In [ ]:
!git clone https://github.com/nicola-decao/s-vae-pytorch.git

fatal: destination path 's-vae-pytorch' already exists and is not an empty directory.


In [ ]:
!python s-vae-pytorch/setup.py install

running install
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
writing hyperspherical_vae.egg-info/PKG-INFO
writing dependency_links to hyperspherical_vae.egg-info/dependency_links.txt
writing requirements to hyperspherical_vae.egg-info/requires.txt
writing top-level names to hyperspherical_vae.egg-info/top_level.txt
reading manifest file 'hyperspherical_vae.egg-info/SOURCES.txt'
writing manifest file 'hyperspherical_vae.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64

In [ ]:
!python s-vae-pytorch/examples/mnist.py

Traceback (most recent call last):
  File "/content/s-vae-pytorch/examples/mnist.py", line 11, in <module>
    from hyperspherical_vae.distributions import VonMisesFisher
ModuleNotFoundError: No module named 'hyperspherical_vae'


In [ ]:
import sys as sys
print(sys.path)

sys.path.append("/content/s-vae-pytorch")

['/content', '/env/python', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.9/dist-packages/IPython/extensions', '/root/.ipython', '/content/s-vae-pytorch']


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets, transforms
from collections import defaultdict

from hyperspherical_vae.distributions import VonMisesFisher
from hyperspherical_vae.distributions import HypersphericalUniform

Obtain the batch free single cell data

In [ ]:
!mkdir data
!wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O data/pbmc3k_filtered_gene_bc_matrices.tar.gz
!cd data; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz
!mkdir write 

mkdir: cannot create directory ‘data’: File exists
--2023-04-06 06:13:46--  http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.1.173, 104.18.0.173, 2606:4700::6812:1ad, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz [following]
--2023-04-06 06:13:46--  https://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7621991 (7.3M) [application/x-tar]
Saving to: ‘data/pbmc3k_filtered_gene_bc_matrices.tar.gz’

data/pbmc3k_filtere 100%[===================>]   7.27M  9.52MB/s    in 0.8s    

2023-04-06 06:13:4

In [ ]:
def read_mtx(filename, dtype='int32'):
    from scipy.io import mmread

    x = mmread(filename).astype(dtype)

    return x

In [ ]:
matrix_path = "/content/data/filtered_gene_bc_matrices/hg19/matrix.mtx"
X = read_mtx(matrix_path)
print(X.shape)
X = X.transpose().todense()
print(X.shape)

(32738, 2700)
(2700, 32738)


In [ ]:
print(np.sum(X > 1))
print(np.sum(X == 1))
print(np.sum(X < 1))

689509
1597375
86105716


In [ ]:
X = torch.tensor(X)

In [ ]:
X = X.to(torch.float32)

In [ ]:
class Multiply(nn.Module):
  def __init__(self, library_size):
    super(Multiply, self).__init__()
    self.library_size = library_size
  
  def forward(self, x):
      x *= self.library_size
      return x

In [ ]:
class ModelVAE(torch.nn.Module):
    
    def __init__(self, h_dim, z_dim, activation=F.relu, distribution='normal', x=None):
        """
        ModelVAE initializer
        :param h_dim: dimension of the hidden layers
        :param z_dim: dimension of the latent representation
        :param activation: callable activation function
        :param distribution: string either `normal` or `vmf`, indicates which distribution to use
        """
        super(ModelVAE, self).__init__()
        
        self.z_dim, self.activation, self.distribution = z_dim, activation, distribution
        
        # 2 hidden layers encoder
        # Old dimension for MNIST
        # self.fc_e0 = nn.Linear(784, h_dim * 2)
        # New dimension of input
        self.fc_e0 = nn.Linear(32738, h_dim * 2)
        self.fc_e1 = nn.Linear(h_dim * 2, h_dim)

        if self.distribution == 'normal':
            # compute mean and std of the normal distribution
            self.fc_mean = nn.Linear(h_dim, z_dim)
            self.fc_var =  nn.Linear(h_dim, z_dim)
        elif self.distribution == 'vmf':
            # compute mean and concentration of the von Mises-Fisher
            self.fc_mean = nn.Linear(h_dim, z_dim)
            self.fc_var = nn.Linear(h_dim, 1)
        else:
            raise NotImplemented
            
        # 2 hidden layers decoder
        self.fc_d0 = nn.Linear(z_dim, h_dim)
        self.fc_d1 = nn.Linear(h_dim, h_dim * 2)
        # Old dimension for MNIST
        # self.fc_logits = nn.Linear(h_dim * 2, 784)
        # New dimension of input
        self.fc_logits = nn.Linear(h_dim * 2, 32738)
        self.mu_layer = nn.Linear(h_dim * 2, 32738)
        self.var_layer = nn.Linear(h_dim * 2, 32738)

        # self.library_size = torch.sum(x[0: 64,:], dim=1, keepdim=True)

    def encode(self, x):
        # 2 hidden layers encoder
        x = self.activation(self.fc_e0(x))
        x = self.activation(self.fc_e1(x))
        
        if self.distribution == 'normal':
            # compute mean and std of the normal distribution
            z_mean = self.fc_mean(x)
            z_var = F.softplus(self.fc_var(x))
        elif self.distribution == 'vmf':
            # compute mean and concentration of the von Mises-Fisher
            z_mean = self.fc_mean(x)
            z_mean = z_mean / z_mean.norm(dim=-1, keepdim=True)
            # the `+ 1` prevent collapsing behaviors
            z_var = F.softplus(self.fc_var(x)) + 1
        else:
            raise NotImplemented
        
        return z_mean, z_var
        
    def decode(self, z):
        
        x = self.activation(self.fc_d0(z))
        x = self.activation(self.fc_d1(x))
        # x = self.fc_logits(x)
        mu = F.softmax(self.mu_layer(x))
        # mu *= self.library_size
        # multiply = Multiply(library_size)
        # mu.requires_grad= True
        # mu = multiply(mu)
        sigma_square = F.softplus(self.var_layer(x))
        sigma_square = torch.sum(sigma_square, dim=0)
        return mu, sigma_square
        
    def reparameterize(self, z_mean, z_var):
        if self.distribution == 'normal':
            q_z = torch.distributions.normal.Normal(z_mean, z_var)
            p_z = torch.distributions.normal.Normal(torch.zeros_like(z_mean), torch.ones_like(z_var))
        elif self.distribution == 'vmf':
            q_z = VonMisesFisher(z_mean, z_var)
            p_z = HypersphericalUniform(self.z_dim - 1)
        else:
            raise NotImplemented

        return q_z, p_z
        
    def forward(self, x): 
        z_mean, z_var = self.encode(x)
        q_z, p_z = self.reparameterize(z_mean, z_var)
        z = q_z.rsample()
        # x_ = self.decode(z)
        mu_, sigma_square_ = self.decode(z)
        return (z_mean, z_var), (q_z, p_z), z, mu_, sigma_square_

In [ ]:
def log_likelihood(model, x, n=10):
    """
    :param model: model object
    :param optimizer: optimizer object
    :param n: number of MC samples
    :return: MC estimate of log-likelihood
    """
    # Old dimension for MNIST
    # z_mean, z_var = model.encode(x.reshape(-1, 784))
    z_mean, z_var = model.encode(x)
    q_z, p_z = model.reparameterize(z_mean, z_var)
    z = q_z.rsample(torch.Size([n]))
    x_mb_ = model.decode(z)

    log_p_z = p_z.log_prob(z)

    if model.distribution == 'normal':
        log_p_z = log_p_z.sum(-1)

    log_p_x_z = -nn.BCEWithLogitsLoss(reduction='none')(x_mb_, x.reshape(-1, 784).repeat((n, 1, 1))).sum(-1)

    log_q_z_x = q_z.log_prob(z)

    if model.distribution == 'normal':
        log_q_z_x = log_q_z_x.sum(-1)

    return ((log_p_x_z + log_p_z - log_q_z_x).t().logsumexp(-1) - np.log(n)).mean()

In [ ]:
def log_likelihood_nb(x, mu, sigma, eps=1e-16):

    log_mu_sigma = torch.log(mu + sigma + eps)

    ll = torch.lgamma(x + sigma) - torch.lgamma(sigma) - \
        torch.lgamma(x + 1) + sigma * torch.log(sigma + eps) - \
        sigma * log_mu_sigma + x * torch.log(mu + eps) - x * log_mu_sigma

    return ll

In [ ]:
library_size = None
def train(model, optimizer):
    # for i, (x_mb, y_mb) in enumerate(train_loader):
    for i in range(0, X.shape[0], 64):
    # for x in X:
            if i+64 < X.shape[0]:
              x = X[i: i+64, :]
            else:
              x = X[i:, :]
            optimizer.zero_grad()
            
            # dynamic binarization
            # x_mb = (x_mb > torch.distributions.Uniform(0, 1).sample(x_mb.shape)).float()

            # _, (q_z, p_z), _, x_mb_ = model(x_mb.reshape(-1, 784))
            global library_size
            library_size = torch.sum(x, dim=1, keepdim=True)
            (z_mean, z_var), (q_z, p_z), _, mu, sigma_square = model(x)
            # loss_recon = nn.BCEWithLogitsLoss(reduction='none')(x_reconstructed, x).sum(-1).mean()
            # loss_recon = log_likelihood(x, z_mean, z_var)
            mu = mu * library_size
            loss_recon = log_likelihood_nb(
                x,
                mu,
                sigma_square,
            ).sum(-1).mean()
          
            if model.distribution == 'normal':
                loss_KL = torch.distributions.kl.kl_divergence(q_z, p_z).sum(-1).mean()
            elif model.distribution == 'vmf':
                loss_KL = torch.distributions.kl.kl_divergence(q_z, p_z).mean()
            else:
                raise NotImplemented

            loss = loss_recon + loss_KL

            loss.backward()
            optimizer.step()

In [ ]:
# hidden dimension and dimension of latent space
H_DIM = 128
Z_DIM = 5
torch.autograd.set_detect_anomaly(True)
# normal VAE
modelN = ModelVAE(h_dim=H_DIM, z_dim=Z_DIM, distribution='normal', x=X)
optimizerN = optim.Adam(modelN.parameters(), lr=1e-3)

print('##### Normal VAE #####')

# training for 1 epoch
train(modelN, optimizerN)

# test
# test(modelN, optimizerN)

print()

# hyper-spherical  VAE
modelS = ModelVAE(h_dim=H_DIM, z_dim=Z_DIM + 1, distribution='vmf')
optimizerS = optim.Adam(modelS.parameters(), lr=1e-3)

print('##### Hyper-spherical VAE #####')

# training for 1 epoch
train(modelS, optimizerS)

# test
# test(modelS, optimizerS)

##### Normal VAE #####


<ipython-input-84-71ac4c9d4f99>:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  mu = F.softmax(self.mu_layer(x))



##### Hyper-spherical VAE #####


/usr/local/lib/python3.9/dist-packages/torch/distributions/distribution.py:51: UserWarning: <class 'hyperspherical_vae.distributions.hyperspherical_uniform.HypersphericalUniform'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [ ]:
X.dtype

torch.float32